In [2]:
import sys
import zmq

REQUEST_TIMEOUT = 2500
REQUEST_RETRIES = 3
SERVER_ENDPOINT = "tcp://localhost:5555"

context = zmq.Context(1)

print "I: Connecting to server…"
client = context.socket(zmq.REQ)
client.connect(SERVER_ENDPOINT)

poll = zmq.Poller()
poll.register(client, zmq.POLLIN)

sequence = 0
retries_left = REQUEST_RETRIES
while retries_left:
    sequence += 1
    request = str(sequence)
    print "I: Sending (%s)" % request
    client.send(request)

    expect_reply = True
    while expect_reply:
        socks = dict(poll.poll(REQUEST_TIMEOUT))
        if socks.get(client) == zmq.POLLIN:
            reply = client.recv()
            if not reply:
                break
            if int(reply) == sequence:
                print "I: Server replied OK (%s)" % reply
                retries_left = REQUEST_RETRIES
                expect_reply = False
            else:
                print "E: Malformed reply from server: %s" % reply

        else:
            print "W: No response from server, retrying…"
            # Socket is confused. Close and remove it.
            client.setsockopt(zmq.LINGER, 0)
            client.close()
            poll.unregister(client)
            retries_left -= 1
            if retries_left == 0:
                print "E: Server seems to be offline, abandoning"
                break
            print "I: Reconnecting and resending (%s)" % request
            # Create new connection
            client = context.socket(zmq.REQ)
            client.connect(SERVER_ENDPOINT)
            poll.register(client, zmq.POLLIN)
            client.send(request)

context.term()

I: Connecting to server…
I: Sending (1)


ValueError: invalid literal for int() with base 10: 'World'

In [1]:
## Hello World


import zmq

context = zmq.Context()

#  Socket to talk to server
print("Connecting to hello world server…")
socket = context.socket(zmq.REQ)
socket.connect("tcp://localhost:5555")

#  Do 10 requests, waiting each time for a response
for request in range(10):
    print("Sending request %s …" % request)
    socket.send(b"Hello")

    #  Get the reply.
    message = socket.recv()
    print("Received reply %s [ %s ]" % (request, message))

Connecting to hello world server…
Sending request 0 …
Received reply 0 [ World ]
Sending request 1 …
Received reply 1 [ World ]
Sending request 2 …
Received reply 2 [ World ]
Sending request 3 …
Received reply 3 [ World ]
Sending request 4 …
Received reply 4 [ World ]
Sending request 5 …
Received reply 5 [ World ]
Sending request 6 …
Received reply 6 [ World ]
Sending request 7 …
Received reply 7 [ World ]
Sending request 8 …
Received reply 8 [ World ]
Sending request 9 …
Received reply 9 [ World ]


In [ ]:
## Pair Connection

import zmq

context = zmq.Context()

#  Socket to talk to server
print("Connecting to hello world server…")
socket = context.socket(zmq.REQ)
socket.connect("tcp://localhost:5555")


    socket.send("c",16)

    #  Get the reply.
    message = socket.recv()
    print("Received reply %s [ %s ]" % (request, message))